In [7]:
import pandas as pd
from Project import collector

In [8]:
data = collector.single_run_collector()
data[data["Round" == 200]]


step: 0
initiator in comm: 0, receivers in comm: []
0 found no receivers
initiator in comm: 1, receivers in comm: []
1 found no receivers
initiator in comm: 2, receivers in comm: []
2 found no receivers
initiator in comm: 3, receivers in comm: [4]


KeyError: 4